In [3]:
import tensorflow as tf

import tensorflow.keras as keras 
import pandas as pd
import numpy as np
import datetime as dt
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


# Data Loading & Preprocessing

In [4]:
data_train = pd.read_csv('./data/codefest/train_split.csv')
data_valid = pd.read_csv('./data/codefest/val_split.csv')
data_train['date'] = pd.to_datetime(data_train['date'])
data_train['date']=data_train['date'].map(dt.datetime.toordinal)
data_valid['date'] = pd.to_datetime(data_valid['date'])
data_valid['date']=data_valid['date'].map(dt.datetime.toordinal)
y_train = data_train['sales']
X_train = data_train.drop('sales', axis=1)
y_valid = data_valid['sales']
X_valid = data_valid.drop('sales', axis=1)
X_train = X_train.values
y_train = y_train.values
X_valid = X_valid.values
y_valid = y_valid.values
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

# Deep Learning Neural Network Initialization & Training

In [5]:
model = keras.models.Sequential()
model.add(keras.layers.Input((3,)))
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(512, activation='relu', kernel_initializer='he_normal'))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Dense(1, activation='relu', kernel_initializer='he_normal'))

In [6]:
checkpoint_filepath = './model/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.01),
    loss=tf.keras.losses.MeanAbsoluteError(),
    metrics=[tf.keras.metrics.MeanAbsoluteError()],
)

In [68]:
# model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_valid, y_valid), callbacks=[model_checkpoint_callback])

Train on 1762 samples, validate on 312 samples
Epoch 1/20
1762/1762 [==============================] - 3s 2ms/sample - loss: 1.2242 - mean_absolute_error: 1.2242 - val_loss: 1.4881 - val_mean_absolute_error: 1.4881
Epoch 2/20
1762/1762 [==============================] - 1s 467us/sample - loss: 0.7557 - mean_absolute_error: 0.7557 - val_loss: 1.1254 - val_mean_absolute_error: 1.1254
Epoch 3/20
1762/1762 [==============================] - 1s 391us/sample - loss: 0.4825 - mean_absolute_error: 0.4825 - val_loss: 0.7125 - val_mean_absolute_error: 0.7125
Epoch 4/20
1762/1762 [==============================] - 0s 237us/sample - loss: 0.4227 - mean_absolute_error: 0.4227 - val_loss: 1.2151 - val_mean_absolute_error: 1.2151
Epoch 5/20
1762/1762 [==============================] - 1s 416us/sample - loss: 0.4383 - mean_absolute_error: 0.4383 - val_loss: 0.7864 - val_mean_absolute_error: 0.7864
Epoch 6/20
1762/1762 [==============================] - 1s 472us/sample - loss: 0.4268 - mean_absolute_er

In [9]:
model = keras.models.load_model('./model/codefest')

In [10]:
model.evaluate(X_valid, y_valid)

312/312 [==============================] - 0s 363us/sample - loss: 0.5351 - mean_absolute_error: 0.5351


[0.5351433530449867, 0.53514344]

In [11]:
def draw(scaler, model, name=None, df_path = './data/codefest/train.csv', save_fig=0):
    
    df = pd.read_csv(df_path)
    df['date'] = pd.to_datetime(df['date'])
    df.sort_values(by=['date'], inplace=True, ascending=False)
    dates = df['date'].values
    df['date']=df['date'].map(dt.datetime.toordinal)
    y = df['sales'].values
    df = df.drop('sales', axis=1)
    X = df.values
    X = scaler.transform(X)
    pred = model.predict(X)
    loss, _ = model.evaluate(X, y)
    pred_upper = pred+loss
    pred_lower = pred-loss
#     plt.xlabel(dates)
    plt.rcParams['figure.figsize'] = [300,100]
    plt.rcParams.update({'font.size':200})
    pred_upper = np.reshape(pred_upper, (-1,))
    pred_lower = np.reshape(pred_lower, (-1,))
    plt.plot(dates,y, 'ro', ms=40)
    plt.plot(dates, pred, lw=30)
    plt.plot(dates, pred_upper, lw=10, color='b')
    plt.plot(dates, pred_lower, lw=10, color='b')
    plt.fill_between(dates,pred_upper, pred_lower, color='b', alpha=0.2)
    plt.title('codefest')
    
    
    if save_fig==0:
        plt.show()
    else:
        plt.savefig('./codefest_data_plot/codefest', dpi=10, format='jpeg')
        plt.clf()
    

In [12]:
draw(scaler=scaler, model=model)

2074/2074 [==============================] - 0s 43us/sample - loss: 0.3227 - mean_absolute_error: 0.3227


In [ ]:
draw(save_fig=1, scaler=scaler, model=model, name='codefest')